In [7]:
import googlemaps
import pandas as pd
import time
import os 

# save env
import pickle  
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time 
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import csv
from pandas import Series,DataFrame
import unicodecsv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import pandas as pd
import numpy as np
import os 
from selenium.webdriver.chrome.service import Service


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent


import pyautogui
from selenium.webdriver.common.action_chains import ActionChains #move_to_element
from selenium.common.exceptions import StaleElementReferenceException #error > element is not attached to the page document
from selenium.common.exceptions import NoSuchElementException #點完所有按鈕時
from  itertools import permutations
import nltk
import re


import argparse
import pprint
import gensim
from glove import Glove
from glove import Corpus

# %%
company_list = pd.read_excel('D:\\Stata\\stata 桌面\\stata系列\\計畫18_又予\\20210104新計畫\\20210128_重新整理\\以sp1500為主(新的)\\stata\\10-K\\sp1500\\sp1500.xls')

# %% [markdown]
# ## RULE

# %%
############################################  keywords  ############################################
##  keyword list 1 
keywords = ["essential","key","core","important","skill","skilled","skillful","trained","experienced","talented","qualified","quality","effective"]

##  keyword list 2
keywords2 = ["worker","employee","eligible employee","personnel","colleague","team","member","individual","people","specialist","labor","staff","professional",
"workforce","professional staff","scientist","technician","research profession","scientific personnel","talent","labor"]

##  keyword list 3
keywords3 = ['recruit and retain','recruit or retain','recruit and retaining','recruit or retaining',
            'recruiting and retain','recruiting or retain','recruiting and retaining',
            'recruiting or retaining','recruit and retain','attract or retain','attract and retain',
            'attract and retaining','attract or retaining','attracting and retain','attracting or retain',
            'attracting and retaining','attracting or retaining','retain and recruit','retain or recruit','retaining and recruit',
            'retaining or recruit','retain and recruiting','retain or recruiting','retaining and recruiting',
            'retaining or recruiting','retain and attract','retain or attract','retaining and attract',
            'retaining or attract','retain and attracting','retain or attracting','retaining and attracting',
            'retaining or attracting']

keywords4 = ["recruit","attract","retain","recruiting","attracting","retaining","attraction","retention","associate","recruitment","motivate","retention","hire"]


##  exclude keyword list
exlude_keywords = ["customer","customers","supplier","suppliers","client","clients","contract","contracts",
                "creditor","creditors","investor","investors","business","businesses","segments","segment",
                "subscriber","subscribers","right","rights"]
##　negative prefix list
negativePrefix = ['un', 'in', 'im','il','ir','non', 'mis','mal','dis','anti','de','under', 'semi', 'mini', 'ex', 'sub', 'infra']

##  keywords list combination (just for keywords1 - 2)
key_combination_list = ['^(?=.*skill).*$']
'''
for k1 in keywords:
    for k2 in keywords2:
        #https://newbedev.com/regex-i-want-this-and-that-and-that-in-any-order
        temp = '^(?=.*'+k1+')(?=.*'+k2+').*$'
        key_combination_list.append(temp)
'''
for k1 in keywords:
    for k2 in keywords2:
        #temp = " ".join([k1,k2])
        #k2+'.*?'+k3
        temp = '^(?=.*'+k1+'.*?'+k2+').*$'
        key_combination_list.append(temp)
for k1 in keywords:
    for k2 in keywords2:
        temp = " ".join([k1,k2])
        #k2+'.*?'+k3
        temp = '^(?=.*'+temp+').*$'
        key_combination_list.append(temp)


##  keywords list combination (for keywords3)
key_combination_list_for_keyword3 = [] 
for k3 in keywords3:
    temp = '^(?=.*'+k3+').*$'
    key_combination_list_for_keyword3.append(temp)

for k1 in keywords4:
    for k2 in keywords4:
        if k1!= k2:
            temp = k1+'.*?'+"and "+k2
            key_combination_list_for_keyword3.append(temp)
            temp = k1+'.*?'+"or "+k2
            key_combination_list_for_keyword3.append(temp)
        else:
            pass

for k1 in keywords4:
    for k2 in keywords4:
        if k1!= k2:
            temp = k1+'.*?'+", "+k2
            key_combination_list_for_keyword3.append(temp)
        else:
            pass
##  exclude  keywords list combination 
#https://learnbyexample.github.io/python-regex-cheatsheet/

##  exclude  keywords list combination (just for keywords1 negative prefix)
exclude_key_combination_list = ['^(?=.*design).*$']
for k1 in negativePrefix:
    for k2 in keywords:
        temp = "".join([k1,k2])
        temp = '^(?=.*'+temp+').*$'
        temp1 = "-".join([k1,k2])
        temp1 = '^(?=.*'+temp1+').*$'
        exclude_key_combination_list.append(temp)
        exclude_key_combination_list.append(temp1)

##  exclude  keywords list combination (just for keywords3)
exclude_key_combination_list_for_keywords3 = ['^(?=.*design).*$']
for k1 in keywords3:
    for k2 in exlude_keywords:
        temp = k1+'.*?'+k2
        exclude_key_combination_list_for_keywords3.append(temp)

for k1 in keywords4:
    for k2 in keywords4:
        for k3 in exlude_keywords:
            if k1!= k2:
                temp = k1+'.*?'+"and "+k2+'.*?'+k3
                exclude_key_combination_list_for_keywords3.append(temp)
                temp = k1+'.*?'+"or "+k2+'.*?'+k3
                exclude_key_combination_list_for_keywords3.append(temp)
            else:
                pass


for k1 in negativePrefix:
    for k2 in keywords3:
        temp = "".join([k1,k2])
        temp = '^(?=.*'+temp+').*$'
        temp1 = "-".join([k1,k2])
        temp1 = '^(?=.*'+temp1+').*$'
        exclude_key_combination_list_for_keywords3.append(temp)
        exclude_key_combination_list_for_keywords3.append(temp1)

##  keywords product list combination 
key_product_combination_list = []
for k2 in keywords2:
    #https://www.regextester.com/15
    temp = '(?=.*product)^((?!'+k2+').)*$'
    key_product_combination_list.append(temp)



############################################  count keywords    ############################################
## count how many sentences containing specific keywords  
def count_keywords_function(sentence_list,keywords_type):
    temp  = []
    for sentence in sentence_list :
        temp.append((sum(1 for 
        word in keywords_type if len(re.findall(word, sentence))==1)))
    return temp

def extract_sentence_function(sentence_list,keywords_location_list,negative):

    temp_sentences  = []
    
    if negative == 1:
        for location in [i for i in range(len(keywords_location_list)) if keywords_location_list[i] == 0]:
            #print(location)
            temp_sentences.append(sentence_list[location])
            #print(sentence_list[location])
    else:
        for location in [i for i in range(len(keywords_location_list)) if keywords_location_list[i] > 0]:
            #print(location)
            if len(sentence_list[location].split())>20:
                temp_sentences.append(sentence_list[location])
            #print(sentences[location])
        
    return temp_sentences

def extract_sentence_function_for_design(sentence_list,negative):
    temp_sentences  = []
    keywords_location_list  = []
    keywords_type_list = ['^(?=.*design.*?employee).*$','^(?=.*design.*?talent).*$']
    for sentence in sentence_list :
        keywords_location_list.append((sum(1 for word in keywords_type_list if len(re.findall(word, sentence))==1)))
    if negative == 1:
        for location in [i for i in range(len(keywords_location_list)) if keywords_location_list[i]>0]:
            #print(location)
            temp_sentences.append(sentence_list[location])
            #print(sentence_list[location])
    else:
        pass
    return temp_sentences

In [9]:
parent_dir = "D:/Stata/stata 桌面/stata系列/計畫18_又予/20210104新計畫/20210128_重新整理/以sp1500為主(新的)/stata/10-K/10-K報表"
import pdftotext
for z in range(58,59):
    print("第幾個",z,"\n哪家公司",company_list['company name'].iloc[z])
    parent_dir = "D:/Stata/stata 桌面/stata系列/計畫18_又予/20210104新計畫/20210128_重新整理/以sp1500為主(新的)/stata/10-K/10-K報表"
    directory = str(company_list['tic'].iloc[z])
    path = os.path.join(parent_dir, directory) 
    try:
        os.chdir(path)
        list_in_file=os.listdir(path)
        #list_in_file = [int(x.split(str(company_list['company name'].iloc[z]))[1].split('.pdf')[0]) for x in list_in_file]

        ############################################  read pdf  ############################################
        ## read package
        import pdftotext
        from six.moves.urllib.request import urlopen
        skill_labor_risk_company = pd.DataFrame()
        error_year = []
        #max_lst = max(list_in_file)+1
        for i in os.listdir(path):
            pdf_year=i
            print(pdf_year)
            try:
                ## open pdf
                with open(pdf_year , "rb") as f:
                    pdf = pdftotext.PDF(f)
            except:
                pass
            ## merge all in a list
            paragraph = [] 
            for page in pdf:
                paragraph.append(page)
            # as per recommendation from @freylis, compile once only
            ##  clean string 
            CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 
            def cleanhtml(raw_html):
                cleantext = re.sub(CLEANR, '', raw_html)
                return cleantext
            ## turn to lower    
            paragraph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph]
            paragraph = [cleanhtml(x) for x in paragraph]



            ############################################  Deal with Item paragraph  ############################################
            ##  find Item paragraph
            try:
                pdf_year = i.split(str(company_list['company name'].iloc[z]))[1]
            except:
                pdf_year = i.split(str(company_list['tic'].iloc[z]))[1]
            
            try:
                # item 1.
                temp = [x.find('item 1. ') for x in paragraph]
                # item 2. or item 1a.
                if int(pdf_year.split(".")[0])<=2004:
                    temp2 = [x.find('item 2. ') for x in paragraph]
                    
                else:
                    temp2 = [x.find('item 1a. ') for x in paragraph]
                    temp2_plus = [x.find('item 2. ') for x in paragraph]
                # item 3.
                temp3 = [x.find('item 7. ') for x in paragraph]
                # item 4.
                temp4 = [x.find('item 8. ') for x in paragraph]
            except:
                # item 1.
                temp = [x.find('item1. ') for x in paragraph]
                # item 2. or item 1a.
                if int(pdf_year.split(".")[0])<=2004:
                    temp2 = [x.find('item2. ') for x in paragraph]
                    
                else:
                    temp2 = [x.find('item1a. ') for x in paragraph]
                    temp2_plus = [x.find('item2. ') for x in paragraph]
                # item 3.
                temp3 = [x.find('item7. ') for x in paragraph]
                # item 4.
                temp4 = [x.find('item8. ') for x in paragraph]


            
            # split paragraph - item 1 & item 1a
            if int(pdf_year.split(".")[0])<=2004:
                try :
                    item1_paragrph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp.index([x for x in temp if x != -1][0]):temp2.index([x for x in temp2 if x != -1][0])]
                except:
                    error_year.append(pdf_year.split(".")[0])
            else:
                try:    
                    try:
                        item1_paragrph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp.index([x for x in temp if x != -1][1]):temp2.index([x for x in temp2 if x != -1][1])]
                        item1a_paragrph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp2.index([x for x in temp2 if x != -1][1]):temp2_plus.index([x for x in temp2_plus if x != -1][1])]
                        item1_paragrph.extend(item1a_paragrph)
                    except (IndexError):
                        try:
                            item1_paragrph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp.index([x for x in temp if x != -1][0]):temp2_plus.index([x for x in temp2_plus if x != -1][0])]
                        except:
                            error_year.append(pdf_year.split(".")[0])
                except:
                    try:
                        item1_paragrph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp.index([x for x in temp if x != -1][0]):temp2.index([x for x in temp2 if x != -1][0])]
                        item1a_paragrph = [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp2.index([x for x in temp2 if x != -1][0]):temp2_plus.index([x for x in temp2_plus if x != -1][0])]
                        item1_paragrph.extend(item1a_paragrph)
                    except:
                        error_year.append(pdf_year.split(".")[0])      
            try:      
                # split paragraph - item 7
                item7_paragrph= [x.lower().replace("\n", " ").replace("\r", " ") for x in paragraph][temp3.index([x for x in temp3 if x != -1][0]):temp4.index([x for x in temp4 if x != -1][0])]

                # merge  list
                item1_paragrph.extend(item7_paragrph)
                # one paragraph
                item1_paragrph = ' '.join(item1_paragrph)
            except:
                error_year.append(pdf_year.split(".")[0])

            ############################################  using lemmatize_sentence to transform paragraph  ############################################
            from pywsd.utils import lemmatize_sentence
            # split one paragraph into sentences
            nltk.download('punkt')
            sent_segmenter = nltk.data.load('tokenizers/punkt/english.pickle')
            #sentences =  item1_paragrph.split(". ")
            #sentences = [x.replace("”","") for  x in sentences]
            try:
                sentences = sent_segmenter.tokenize(item1_paragrph)
                sentences = [' '.join(lemmatize_sentence(y)) for y in sentences]
                sentences = [' '.join(y.split(";")) for y in sentences]
            except:
                #sentences = [str(y) for y in item1_paragrph]
                item1_paragrph = [sent_segmenter.tokenize(str(y)) for y in item1_paragrph]
                sentences = sum(item1_paragrph, [])
                sentences = [' '.join(lemmatize_sentence(y)) for y in sentences]
                sentences = [' '.join(y.split(";")) for y in sentences]
                pass
        

            ############################################  count keywords    ############################################

            ## without excluding some keywords (keywords1 - 2 skill labor risk)
            temp = count_keywords_function(sentences,key_combination_list)
            temp_sentences = extract_sentence_function(sentences,temp,0)

            ## exclude some unnecessary keywords (keywords1 - 2 skill labor risk)
            temp = count_keywords_function(temp_sentences,exclude_key_combination_list)
            temp_sentences_exclude = extract_sentence_function(temp_sentences,temp,1)
            temp_sentences_exclude_design = extract_sentence_function_for_design(temp_sentences,1)
            temp_sentences_exclude.extend(temp_sentences_exclude_design)
            #temp = count_keywords_function(temp_sentences_exclude ,exclude_key_combination_list_for_keywords3)
            #temp_sentences_exclude = extract_sentence_function(temp_sentences_exclude ,temp,1)

            ## without excluding some keywords (keywords 3 skill labor risk)
            temp_keywords3 = count_keywords_function(sentences,key_combination_list_for_keyword3)
            temp_sentences_keywords3 = extract_sentence_function(sentences,temp_keywords3 ,0)

            ## exclude some unnecessary keywords (keywords 3 skill labor risk)
            temp_keywords3 = count_keywords_function(temp_sentences_keywords3,exclude_key_combination_list_for_keywords3)
            temp_sentences_exclude_keywords3 = extract_sentence_function(temp_sentences_keywords3,temp_keywords3,1)
        
            #temp_keywords3 = count_keywords_function(temp_sentences_exclude_keywords3 ,exclude_key_combination_list)
            #temp_sentences_exclude_keywords3 = extract_sentence_function(temp_sentences_exclude_keywords3,temp_keywords3,1)


            ## combine keyword1、2、3 skill labor risk
            temp_sentences_exclude.extend(temp_sentences_exclude_keywords3)

            #temp_keywords_1 = count_keywords_function(temp_sentences_exclude ,exclude_key_combination_list)
            #temp_sentences_exclude = extract_sentence_function(temp_sentences_exclude,temp_keywords_1,1)

            ## find sentence where contain "product" and exclude "labor、employee...." which mentioned people
            temp = count_keywords_function(temp_sentences_exclude,key_product_combination_list)
            temp = [1 if x >0 else 0  for x in temp ]

            ############################################  combine to be a dataframe    ############################################
            if temp == [] :
                try:
                    df=pd.DataFrame([[],[],[0],[len(sentences)],[str(i.split(str(company_list['company name'].iloc[z]))[1].split('.pdf')[0])]]).T
                except:
                    df=pd.DataFrame([[],[],[0],[len(sentences)],[str(i.split(str(company_list['tic'].iloc[z]))[1].split('.pdf')[0])]]).T
                df.columns = ["skill_labo_risk_sentence","sentence_not_mention_people","total_sentence_mention_skill_labor_risk","total_sentence","Year"]
            else:
                try:
                    df=pd.DataFrame([temp_sentences_exclude,temp,[len(temp)]*len(temp),[len(sentences)]*len(temp),[str(i.split(str(company_list['company name'].iloc[z]))[1].split('.pdf')[0])]*len(temp)]).T
                except:
                    df=pd.DataFrame([temp_sentences_exclude,temp,[len(temp)]*len(temp),[len(sentences)]*len(temp),[str(i.split(str(company_list['tic'].iloc[z]))[1].split('.pdf')[0])]*len(temp)]).T
                df.columns = ["skill_labo_risk_sentence","sentence_not_mention_people","total_sentence_mention_skill_labor_risk","total_sentence","Year"]


            skill_labor_risk_company =  skill_labor_risk_company.append(df, ignore_index=True)

        skill_labor_risk_company.drop_duplicates(subset=['skill_labo_risk_sentence', 'Year'], keep="first",inplace=True)
        os.chdir("D:\\Stata\\stata 桌面\\stata系列\\計畫18_又予\\20210104新計畫\\20210128_重新整理\\以sp1500為主(新的)\\stata\\10-K\\10-k skill_labor_risk\\20220207")
        skill_labor_risk_company.to_csv("skill_labor_risk_"+str(company_list['tic'].iloc[z])+"_20220126.csv",encoding='utf-8-sig', index=False)
    except:
        pass


第幾個 58 
哪家公司 APACHE OFFSHORE INVESTMENT PARTNERSHIP
APA1997.pdf


[nltk_data] Downloading package punkt to C:\Users\Sandy
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


APA1998.pdf


[nltk_data] Downloading package punkt to C:\Users\Sandy
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


APA1999.pdf


...

[nltk_data] Downloading package punkt to C:\Users\Sandy
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


APA2019.pdf


[nltk_data] Downloading package punkt to C:\Users\Sandy
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


APA2020.pdf


[nltk_data] Downloading package punkt to C:\Users\Sandy
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
skill_labor_risk_company

,skill_labo_risk_sentence,sentence_not_mention_people,total_sentence_mention_skill_labor_risk,total_sentence,Year
0,None,None,0,178,1993
1,None,None,0,161,1994
2,None,None,0,170,1995
3,the time have collective bargaining agreement ...,1,1,197,1996
4,the time have collective bargaining agreement ...,1,1,182,1997
...,...,...,...,...,...
116,compensation and benefit we offer a comprehens...,0,21,374,2020
117,our ability to attract and retain our user be ...,1,21,374,2020
118,"in addition , we have implement and may contin...",0,21,374,2020
119,"these dynamic have affect , and will likely co...",0,21,374,2020
